## Step 1: Setup

**Explanation:**
We load the `day1_prd.md` artifact from Day 1. This document is the single source of truth for our project's requirements and provides the essential context for the LLM to generate a relevant and accurate database schema.

In [6]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Current working directory: {os.getcwd()}")
print(f"Project root directory: {project_root}")

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, recommended_models_table, prompt_enhancer

# Initialize separate LLM clients for different artifacts to use the latest models from different providers.
# - Schema generation uses a strong instruction-following model
# - Seed data generation uses a model tuned for data generation
schema_client, schema_model_name, schema_api_provider = setup_llm_client(model_name="gemini-2.5-pro")
seed_client, seed_model_name, seed_api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD
prd_content = load_artifact("artifacts/project_prd.md")
if not prd_content:
    print("Warning: Could not load prd_content = artifacts/project_prd.md. Lab may not function correctly.")

Current working directory: c:\Users\labadmin\Documents\220372-AG-AISOFTDEV-Team-2-CodeVoyagers
Project root directory: c:\Users\labadmin


2025-11-06 09:54:03,208 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None
2025-11-06 09:54:04,240 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None
2025-11-06 09:54:04,240 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


### STEP 2 - Generating the SQL Schema (SQLite)

**Explanation:**
This prompt instructs the LLM to act as a Database Administrator (DBA) and generate a SQL schema specifically compatible with SQLite.

Guidance for the LLM and the developer:
- Output only `CREATE TABLE` statements and associated `CREATE INDEX` statements where helpful. Do not include any surrounding markdown fences or explanatory text in the SQL output.
- Use SQLite-compatible types and conventions: prefer `INTEGER`, `TEXT`, `REAL`, `BLOB`, and `NUMERIC`. For auto-incrementing primary keys use `INTEGER PRIMARY KEY AUTOINCREMENT`. Do NOT use `SERIAL`, `BIGSERIAL`, `AUTO_INCREMENT`, or PostgreSQL/MySQL-specific types or DDL.
- Avoid features not supported by SQLite such as `ALTER TABLE ... DROP COLUMN`, `CHECK` constraints that reference subqueries, or advanced index types. Keep DDL portable for SQLite's capabilities.
- Use `FOREIGN KEY` clauses only where appropriate; remember that SQLite enforces foreign keys only when `PRAGMA foreign_keys = ON` is set by the application.
- Provide sensible column constraints (`NOT NULL`, `UNIQUE`) and default values using SQLite-supported expressions.

We will post-process the LLM response with `clean_llm_output(..., language='sql')` to strip markdown and save the pure SQL to `artifacts/schema.sql`. The notebook later uses `cursor.executescript()` to run the SQL, so ensure the output is a single SQL script containing multiple statements separated by semicolons.

In [ ]:
schema_prompt = f"""
You are a senior Database Administrator. Based on the provided PRD context below, generate a complete SQL 
schema that is fully compatible with SQLite.

Requirements:
- Output only valid SQLite DDL statements (e.g., CREATE TABLE, CREATE INDEX). Do NOT include any surrounding markdown, commentary, or explanation; output raw SQL only.
- Use SQLite data types and conventions: INTEGER, TEXT, REAL, BLOB, NUMERIC. For auto-incrementing primary keys use `INTEGER PRIMARY KEY AUTOINCREMENT`.
- Do NOT use PostgreSQL/MySQL-specific types or keywords such as SERIAL, BIGSERIAL, AUTO_INCREMENT, or `ENGINE=` options.
- Avoid features unsupported by SQLite (e.g., ALTER TABLE ... DROP COLUMN, advanced index types). Keep the DDL runnable by SQLite's `sqlite3` and via Python's `cursor.executescript()`.
- Include sensible NOT NULL, UNIQUE constraints and FOREIGN KEY clauses where appropriate. Note: application must enable foreign key enforcement via `PRAGMA foreign_keys = ON`.
- Produce CREATE INDEX statements for columns frequently used in WHERE or JOIN clauses if helpful.
- Ensure the output is a single SQL script with statements separated by semicolons.

PRD CONTEXT:
<prd>
{prd_content}
</prd>

Now generate the SQLite-compatible SQL schema.
"""

print("--- Generating SQL Schema ---")
if prd_content:
     # Enhance the raw schema prompt using the project's prompt enhancer
     enhanced_schema_prompt = prompt_enhancer(schema_prompt)
     print("Schema Enhanced prompt\n", enhanced_schema_prompt)

     # Send the enhanced prompt to the schema-specific LLM client
     generated_schema = get_completion(enhanced_schema_prompt, schema_client, schema_model_name, schema_api_provider)

     # Clean up the generated schema
     cleaned_schema = clean_llm_output(generated_schema, language='sql')
     print(cleaned_schema)

     # Save the cleaned schema to a file
     save_artifact(cleaned_schema, "artifacts/schema.sql", overwrite=True)
else:
     print("Skipping schema generation because PRD is missing.")
     cleaned_schema = ""

print("--- Generating SQL Schema ---")
if prd_content:
    # Enhance the raw schema prompt using the project's prompt enhancer
    enhanced_schema_prompt = prompt_enhancer(schema_prompt)
    print("Schema Enhanced prompt\n", enhanced_schema_prompt)

    # Send the enhanced prompt to the schema-specific LLM client
    generated_schema = get_completion(enhanced_schema_prompt, schema_client, schema_model_name, schema_api_provider)

    # Clean up the generated schema
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)

    # Save the cleaned schema to a file
    save_artifact(cleaned_schema, "artifacts/schema.sql", overwrite=True)
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---


2025-11-06 09:54:04,658 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


### Step 3 - Generating Realistic Seed Data

**Explanation:**
An empty database isn't very useful for development. In Step 2 we generated a SQLite schema and saved it to `artifacts/schema.sql` (and the cleaned SQL is available in the `cleaned_schema` variable).

This step asks the LLM to produce realistic, referentially-consistent seed data as raw SQL `INSERT` statements that can be executed against that schema. The seed data generator MUST:

- Inspect the provided SQL schema (the `<schema>` block below and `artifacts/schema.sql`) and use the exact table and column names from it.
- Respect column types, `NOT NULL` and `UNIQUE` constraints, and any `FOREIGN KEY` relationships declared in the schema. If a primary key is defined as `INTEGER PRIMARY KEY AUTOINCREMENT`, the model may insert `NULL` for the PK and then reference the assigned PK values consistently for foreign keys, or insert explicit numeric IDs — but all foreign key references must remain valid within the generated script.
- Wrap the generated inserts in a transaction (for example, `BEGIN; ... COMMIT;`) to ensure atomic seeding and easier rollback during development.
- Output only raw SQL (no markdown fences or explanatory text).
- Provide realistic, non-sensitive sample values (plausible names, emails, dates, statuses) that match the project's PRD context.

The notebook will save the cleaned SQL to `artifacts/seed_data.sql` and then apply it to the database file in the next step.

In [ ]:
seed_data_prompt = f"""
You are a data specialist. Based on the provided PRD and the exact SQL schema below, generate realistic SQL statements to seed the database for the onboarding tool.

Requirements:
- Inspect the provided SQL schema carefully (the `<schema>` block below and the file `artifacts/schema.sql`) and use the exact table and column names found there.
- Produce a single SQL script that begins with `BEGIN;` and ends with `COMMIT;` to wrap all inserts in a transaction.
- Ensure all `INSERT` statements respect column types, `NOT NULL` and `UNIQUE` constraints, and `FOREIGN KEY` relationships declared in the schema. If primary keys are `INTEGER PRIMARY KEY AUTOINCREMENT`, you may insert `NULL` for those columns and then reference the assigned values consistently for foreign keys, or insert explicit IDs — but relationships must be internally consistent.
- Output only raw SQL (no markdown, no explanatory text).
- Generate between 5 and 10 meaningful `INSERT` statements per major entity group (for example: users/project_managers/employees/applicants/onboarding_tasks) so the dataset is useful for development and testing.
- Include at least 5 project managers and 3 employees, and a spread of onboarding tasks assigned to those users. Use realistic values for names, emails, dates (ISO 8601), and status fields consistent with the PRD.

**PRD Context:**
<prd>
{prd_content}
</prd>

**SQL Schema:**
<schema>
{cleaned_schema}
</schema>
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    # Enhance the seed data prompt for better structure and fidelity
    enhanced_seed_prompt = prompt_enhancer(seed_data_prompt)
    print("Seed Data Enhanced prompt\n", enhanced_seed_prompt)

    # Use the seed-data specific client
    generated_seed_data = get_completion(enhanced_seed_prompt, seed_client, seed_model_name, seed_api_provider)

    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)

    # Save the cleaned seed data to a file
    save_artifact(cleaned_seed_data, "artifacts/seed_data.sql", overwrite=True)
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---


2025-11-06 09:51:22,336 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


Seed Data Enhanced prompt
 <prompt>
  <persona>
    You are a senior database engineer and data-seeding specialist with expert knowledge of SQL, relational‐data integrity, and realistic test-data generation.
  </persona>

  <context>
    1. Product domain: “TalentSphere AI” – an internal talent‐mobility platform for Project Managers, HR partners, and Team Leads.  
    2. Functional expectations (selected from PRD): user/manager onboarding, employee profiles, skills, projects, performance reviews, AI conversations, etc.  
    3. Exact database schema (SQLite dialect, copied verbatim below).  
       <schema>
CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    created_at TEXT NOT NULL DEFAULT (datetime('now'))
);
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    job_title TEXT NOT NULL,
    department_id IN

### Step 4 - Creating and Seeding a Live Database

**Explanation:**
This Python function demonstrates a crucial engineering task: turning text-based artifacts into a live system component. The `create_database` function uses Python's built-in `sqlite3` library.
1.  It establishes a connection to a database file, which creates the file if it doesn't exist.
2.  It reads the `schema.sql` artifact and executes it. It's important to use `cursor.executescript()` here. While `cursor.execute()` is designed for a single SQL statement, `executescript()` is necessary for running a string that contains multiple SQL statements, which is exactly what our `schema.sql` and `seed_data.sql` files contain.
3.  It then reads and executes the `seed_data.sql` artifact to populate the newly created tables.
4.  `conn.commit()` saves all the changes to the database file.
5.  The `finally` block ensures that `conn.close()` is always called, which is a critical best practice to prevent resource leaks.

In [ ]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return

    # Delete the old database file if it exists to start fresh
    if os.path.exists(db_path):
        os.remove(db_path)
        print(f"Removed existing database file at {db_path}")

    conn = None
    try:
        conn = sqlite3.connect(db_path)
        # Enable foreign key enforcement for this connection
        conn.execute("PRAGMA foreign_keys = ON")
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # Read and execute the schema file
        schema_sql = load_artifact(schema_path)
        if schema_sql:
            cursor.executescript(schema_sql)
            print("Tables created successfully.")

        # Read and execute the seed data file if it exists
        if os.path.exists(seed_path):
            seed_sql = load_artifact(seed_path)
            if seed_sql:
                cursor.executescript(seed_sql)
                print("Seed data inserted successfully.")

        conn.commit()
        print("Database changes committed.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

# Define file paths
db_file = os.path.join(os.getcwd(), "artifacts", "main_database.db")
schema_file = os.path.join(os.getcwd(), "artifacts", "schema.sql")
seed_file = os.path.join(os.getcwd(), "artifacts", "seed_data.sql")

# Execute the function to create and seed the database
create_database(db_file, schema_file, seed_file)



Successfully connected to database at c:\Users\labadmin\Documents\220372-AG-AISOFTDEV-Team-2-CodeVoyagers\artifacts\main_database.db
Tables created successfully.
Seed data inserted successfully.
Database changes committed.
Tables created successfully.
Seed data inserted successfully.
Database changes committed.


### Step 5 - Verify the database was created successfully by querying the data


In [ ]:
# Verify the database was created successfully by querying the data
def verify_database(db_path, schema_path=None):
    """Verify the database contains the expected data based on the generated schema."""
    if not os.path.exists(db_path):
        print(f"Database file not found at {db_path}")
        return

    conn = None
    try:
        conn = sqlite3.connect(db_path)
        # Ensure foreign keys are enforced for verification queries
        conn.execute("PRAGMA foreign_keys = ON")
        cursor = conn.cursor()
        print(f"Connected to database at {db_path}")

        # Determine expected tables from the provided schema if available
        schema_sql = ""
        if schema_path and os.path.exists(schema_path):
            schema_sql = load_artifact(schema_path) or ""
        elif 'cleaned_schema' in globals() and cleaned_schema:
            schema_sql = cleaned_schema

        table_names = []
        if schema_sql:
            import re
            # Extract table names from CREATE TABLE statements
            matches = re.findall(r'CREATE\s+TABLE\s+(?:IF\s+NOT\s+EXISTS\s+)?["`]?(\w+)["`]?', schema_sql, flags=re.IGNORECASE)
            table_names = list(dict.fromkeys(matches))  # preserve order, remove duplicates

        # Fallback to common expected tables if schema parsing found nothing
        if not table_names:
            table_names = ['users', 'applicants', 'onboarding_tasks']

        # Query each table for counts and a small sample of rows
        for t in table_names:
            try:
                cursor.execute(f"SELECT COUNT(*) FROM {t}")
                count = cursor.fetchone()[0]
                print(f"Table '{t}' contains {count} records")

                # Get column names to present a representative sample row as a dict
                cursor.execute(f"PRAGMA table_info({t})")
                cols = [r[1] for r in cursor.fetchall()]
                if cols:
                    sel_cols = cols[:4]  # limit number of displayed columns
                    cursor.execute(f"SELECT {', '.join(sel_cols)} FROM {t} LIMIT 3")
                    rows = cursor.fetchall()
                    if rows:
                        print(f"--- Sample rows from {t} ---")
                        for row in rows:
                            # Align values with column names for readability
                            sample = dict(zip(sel_cols, row))
                            print(sample)
            except sqlite3.Error as e:
                print(f"Could not query table {t}: {e}")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

# Verify the database using the generated schema when available
verify_database(db_file, schema_file)

Connected to database at c:\Users\labadmin\Documents\220372-AG-AISOFTDEV-Team-2-CodeVoyagers\artifacts\main_database.db
Table 'departments' contains 5 records
--- Sample rows from departments ---
{'department_id': 1, 'name': 'Engineering', 'created_at': '2025-11-06 14:52:54'}
{'department_id': 2, 'name': 'Product Management', 'created_at': '2025-11-06 14:52:54'}
{'department_id': 3, 'name': 'Human Resources', 'created_at': '2025-11-06 14:52:54'}
Table 'employees' contains 11 records
--- Sample rows from employees ---
{'employee_id': 101, 'first_name': 'Aria', 'last_name': 'Chen', 'email': 'aria.chen@talentsphere.ai'}
{'employee_id': 102, 'first_name': 'Ben', 'last_name': 'Carter', 'email': 'ben.carter@talentsphere.ai'}
{'employee_id': 103, 'first_name': 'Chloe', 'last_name': 'Davis', 'email': 'chloe.davis@talentsphere.ai'}
Table 'users' contains 11 records
--- Sample rows from users ---
{'user_id': 201, 'employee_id': 101, 'sso_subject_id': 'auth0|64f8a9b1c2d3e4f5a6b7c8d9', 'is_active'